In [1]:
from selenium import webdriver 
from bs4 import BeautifulSoup 
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import numpy as np
import platform

if platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname = path).get_name()
    rc('font', family = font_name)
elif platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
else:
    print('Check your OS system')

# covid data crawling

In [2]:
browser = webdriver.Chrome('c:/driver/chromedriver.exe')
results = []

url = "https://www.worldometers.info/coronavirus/#countries" 
browser.get(url)
time.sleep(1)
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

# sorting = browser.find_element_by_xpath('//*[@id="main_table_countries_today"]/thead/tr/th[2]')
# sorting.click()
# time.sleep(3)
# sorting.click() 해도 dataframe 단위에서 index 정렬 다시 해줘야 국가명 오름차순 dataframe 생성 가능

table_list = soup.select('#main_table_countries_today > tbody:nth-child(2) > tr')
# print(table_list) # sorting 이후에 스크롤링해도 국가명으로 나오지 않음

for i in range(0, len(table_list)):
    if table_list[i].select('a.mt_a'):
        country  = table_list[i].select('a.mt_a')[0].text
        tot_cases  = table_list[i].select('td.sorting_1')[0].text
        new_cases  = table_list[i].select('tr > td:nth-child(4)')[0].text
        tot_deaths = table_list[i].select('tr > td:nth-child(5)')[0].text
        new_deaths = table_list[i].select('tr > td:nth-child(6)')[0].text
        tot_recov = table_list[i].select('tr > td:nth-child(7)')[0].text
        new_recov = table_list[i].select('tr > td:nth-child(8)')[0].text
        tests = table_list[i].select('tr > td:nth-child(13)')[0].text
        pop = table_list[i].select('tr > td:nth-child(15)')[0].text
        data = [country, tot_cases, new_cases, tot_deaths, new_deaths, tot_recov, new_recov, tests, pop]
        results.append(data)
    else:
        continue
        
time.sleep(3)
browser.quit()
df = pd.DataFrame(results)
df.columns = ["country", "tot_cases", "new_cases", "tot_deaths", "new_deaths", "tot_recov", "new_recov", "tests", "pop"]
df.set_index("country", inplace=True)
df.sort_index(ascending=True, inplace=True)
df.reset_index(inplace=True)
# df

df.to_excel('./covid19.xlsx', index = False)

# data 전처리

In [51]:
df = pd.read_excel('./covid19.xlsx')
df.head()
df.tail()

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
215,Wallis and Futuna,445,NaN,7,NaN,438,NaN,"20,508","11,035"
216,Western Sahara,10,NaN,1,NaN,8,NaN,NaN,"612,406"
217,Yemen,"6,961",+9,"1,366",NaN,"4,147",+5,"127,681","30,500,337"
218,Zambia,"178,748","+2,006","2,904",+37,"163,681","+1,442","1,977,464","18,911,673"
219,Zimbabwe,"73,271","+2,845","2,274",+38,"48,102","+1,170","896,362","15,085,471"


In [52]:
# column명 변경 필요 없도록 xlsx file 변경함
# df.columns = ["country", "tot_cases", "new_cases", "tot_deaths", "new_deaths", "tot_recov", "new_recov", "tests", "pop"]
# df.head()

In [53]:
# 데이터 속 , 삭제, dtype int로 변경
df['tot_cases'] = df['tot_cases'].str.replace(',', '')
df.head()

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,Afghanistan,136643,+754,"5,921",+72,"83,849","+1,263","664,045","39,828,263"
1,Albania,132608,+11,"2,456",NaN,"130,053",+1,"840,087","2,874,554"
2,Algeria,147883,+941,"3,867",+16,"102,594",+584,"230,861","44,664,792"
3,Andorra,14167,+12,127,NaN,"13,796",+9,"193,595","77,393"
4,Angola,40055,NaN,939,NaN,"34,357",NaN,"669,500","33,913,305"


In [54]:
df['new_cases'] = df['new_cases'].str.replace('+', '').replace(',', '')
# df['new_cases'] = df['new_cases'].str.replace(',', '')
df.head()
# nan값 생김

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,Afghanistan,136643,754,"5,921",+72,"83,849","+1,263","664,045","39,828,263"
1,Albania,132608,11,"2,456",NaN,"130,053",+1,"840,087","2,874,554"
2,Algeria,147883,941,"3,867",+16,"102,594",+584,"230,861","44,664,792"
3,Andorra,14167,12,127,NaN,"13,796",+9,"193,595","77,393"
4,Angola,40055,NaN,939,NaN,"34,357",NaN,"669,500","33,913,305"


In [55]:
df['tot_deaths'] = df['tot_deaths'].str.replace(',', '')
df.head()
# int 변환에 문제생김

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,Afghanistan,136643,754,5921,+72,"83,849","+1,263","664,045","39,828,263"
1,Albania,132608,11,2456,NaN,"130,053",+1,"840,087","2,874,554"
2,Algeria,147883,941,3867,+16,"102,594",+584,"230,861","44,664,792"
3,Andorra,14167,12,127,NaN,"13,796",+9,"193,595","77,393"
4,Angola,40055,NaN,939,NaN,"34,357",NaN,"669,500","33,913,305"


In [56]:
df['new_deaths'] = df['new_deaths'].str.replace('+', '').replace(',', '').replace(' ', '')
# df['new_deaths'] = df['new_deaths'].str.replace(',', '')
# df['new_deaths'] = df['new_deaths'].str.replace(' ', '')
df.head(30)
# nan 값 있음

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,Afghanistan,136643,754,5921,72,"83,849","+1,263","664,045","39,828,263"
1,Albania,132608,11,2456,NaN,"130,053",+1,"840,087","2,874,554"
2,Algeria,147883,941,3867,16,"102,594",+584,"230,861","44,664,792"
3,Andorra,14167,12,127,NaN,"13,796",+9,"193,595","77,393"
4,Angola,40055,NaN,939,NaN,"34,357",NaN,"669,500","33,913,305"
5,Anguilla,111,NaN,,NaN,109,NaN,"24,074","15,141"
6,Antigua and Barbuda,1266,NaN,42,NaN,"1,222",NaN,"16,799","98,764"
7,Argentina,4682960,"20,023",99640,385,"4,314,593","+13,770","17,804,872","45,622,906"
8,Armenia,226597,138,4549,2,"217,810",+79,"1,242,361","2,968,956"
9,Aruba,11169,NaN,107,NaN,"11,029",NaN,"177,885","107,233"


In [29]:
df['tot_recov'] = df['tot_recov'].str.replace(',', '')
df.head()
# cannot convert float NaN to integer

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,Afghanistan,136643,754,5921,72,83849,"+1,263","664,045","39,828,263"
1,Albania,132608,11,2456,NaN,130053,+1,"840,087","2,874,554"
2,Algeria,147883,941,3867,16,102594,+584,"230,861","44,664,792"
3,Andorra,14167,12,127,NaN,13796,+9,"193,595","77,393"
4,Angola,40055,NaN,939,NaN,34357,NaN,"669,500","33,913,305"


In [30]:
df['new_recov'] = df['new_recov'].str.replace('+', '')
df['new_recov'] = df['new_recov'].str.replace(',', '')
df.head()
# int 변환 문제 cannot convert float NaN to integer

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,Afghanistan,136643,754,5921,72,83849,1263,"664,045","39,828,263"
1,Albania,132608,11,2456,NaN,130053,1,"840,087","2,874,554"
2,Algeria,147883,941,3867,16,102594,584,"230,861","44,664,792"
3,Andorra,14167,12,127,NaN,13796,9,"193,595","77,393"
4,Angola,40055,NaN,939,NaN,34357,NaN,"669,500","33,913,305"


In [31]:
df['tests'] = df['tests'].str.replace(',', '')
df.head(60)

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,Afghanistan,136643,754,5921,72,83849,1263,664045,"39,828,263"
1,Albania,132608,11,2456,NaN,130053,1,840087,"2,874,554"
2,Algeria,147883,941,3867,16,102594,584,230861,"44,664,792"
3,Andorra,14167,12,127,NaN,13796,9,193595,"77,393"
4,Angola,40055,NaN,939,NaN,34357,NaN,669500,"33,913,305"
5,Anguilla,111,NaN,,NaN,109,NaN,24074,"15,141"
6,Antigua and Barbuda,1266,NaN,42,NaN,1222,NaN,16799,"98,764"
7,Argentina,4682960,20023,99640,385,4314593,13770,17804872,"45,622,906"
8,Armenia,226597,138,4549,2,217810,79,1242361,"2,968,956"
9,Aruba,11169,NaN,107,NaN,11029,NaN,177885,"107,233"


In [32]:
df['pop'] = df['pop'].str.replace(',', '')
df.head()

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,Afghanistan,136643,754,5921,72,83849,1263,664045,39828263
1,Albania,132608,11,2456,NaN,130053,1,840087,2874554
2,Algeria,147883,941,3867,16,102594,584,230861,44664792
3,Andorra,14167,12,127,NaN,13796,9,193595,77393
4,Angola,40055,NaN,939,NaN,34357,NaN,669500,33913305


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   country     220 non-null    object
 1   tot_cases   220 non-null    object
 2   new_cases   128 non-null    object
 3   tot_deaths  220 non-null    object
 4   new_deaths  99 non-null     object
 5   tot_recov   219 non-null    object
 6   new_recov   117 non-null    object
 7   tests       210 non-null    object
 8   pop         220 non-null    object
dtypes: object(9)
memory usage: 15.6+ KB


In [34]:
df.head()

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,Afghanistan,136643,754,5921,72,83849,1263,664045,39828263
1,Albania,132608,11,2456,NaN,130053,1,840087,2874554
2,Algeria,147883,941,3867,16,102594,584,230861,44664792
3,Andorra,14167,12,127,NaN,13796,9,193595,77393
4,Angola,40055,NaN,939,NaN,34357,NaN,669500,33913305


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   country     220 non-null    object
 1   tot_cases   220 non-null    object
 2   new_cases   128 non-null    object
 3   tot_deaths  220 non-null    object
 4   new_deaths  99 non-null     object
 5   tot_recov   219 non-null    object
 6   new_recov   117 non-null    object
 7   tests       210 non-null    object
 8   pop         220 non-null    object
dtypes: object(9)
memory usage: 15.6+ KB


In [36]:
# 결측치 확인
df.isnull().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   country     220 non-null    object
 1   tot_cases   220 non-null    object
 2   new_cases   128 non-null    object
 3   tot_deaths  220 non-null    object
 4   new_deaths  99 non-null     object
 5   tot_recov   219 non-null    object
 6   new_recov   117 non-null    object
 7   tests       210 non-null    object
 8   pop         220 non-null    object
dtypes: object(9)
memory usage: 15.6+ KB


In [37]:
# 결측치, " "에 0 대입
df = df.replace(np.nan, 0)
df = df.replace(' ', 0)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   country     220 non-null    object
 1   tot_cases   220 non-null    object
 2   new_cases   220 non-null    object
 3   tot_deaths  220 non-null    object
 4   new_deaths  220 non-null    object
 5   tot_recov   220 non-null    object
 6   new_recov   220 non-null    object
 7   tests       220 non-null    object
 8   pop         220 non-null    object
dtypes: object(9)
memory usage: 15.6+ KB


,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,Afghanistan,136643,754,5921,72,83849,1263,664045,39828263
1,Albania,132608,11,2456,0,130053,1,840087,2874554
2,Algeria,147883,941,3867,16,102594,584,230861,44664792
3,Andorra,14167,12,127,0,13796,9,193595,77393
4,Angola,40055,0,939,0,34357,0,669500,33913305
...,...,...,...,...,...,...,...,...,...
215,Wallis and Futuna,445,0,7,0,438,0,20508,11035
216,Western Sahara,10,0,1,0,8,0,0,612406
217,Yemen,6961,9,1366,0,4147,5,127681,30500337
218,Zambia,178748,2006,2904,37,163681,1442,1977464,18911673


In [38]:
# 값에 dtype int 변경
df[['tot_cases', 'new_cases', 'tot_deaths', 'new_deaths', 'tot_recov', 'new_recov', 'tests', 'pop']] = df[['tot_cases', 'new_cases', 'tot_deaths', 'new_deaths', 'tot_recov', 'new_recov', 'tests', 'pop']].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   country     220 non-null    object
 1   tot_cases   220 non-null    int64 
 2   new_cases   220 non-null    int64 
 3   tot_deaths  220 non-null    int64 
 4   new_deaths  220 non-null    int64 
 5   tot_recov   220 non-null    int64 
 6   new_recov   220 non-null    int64 
 7   tests       220 non-null    int64 
 8   pop         220 non-null    int64 
dtypes: int64(8), object(1)
memory usage: 15.6+ KB


# gdp 데이터 정리 전처리

In [39]:
import requests

url='https://www.un.org/en/about-us/member-states'
res = requests.get(url)
html = res.text
soup = BeautifulSoup(html, 'html.parser')

country_names = soup.select('.mb-0')
country_list = []
for i in country_names:
    country_list.append(i.text)

col = ['country']
country_df = pd.DataFrame(country_list, columns=col)

# 영국의 경우 UN에 북아일랜드와 함께 가입되어있는데 gdp 자료는 영국 기준이라서 이름 변경
country_df.iloc[182, 0] = 'United Kingdom'

# 국가별 2018, 2019 gdp 정보가 있는 엑셀 파일 read
gdp_cap = pd.read_excel('gdp_per_capita.xls')
gdp_df = pd.merge(left=country_df,
                right=gdp_cap,
                how='left',
                left_on='country',
                right_on='Country Name')

# df_nulls = gdp_df.isnull().sum()
# un 가입국이지만 gdp 정보가 없는 국가 3개
# 2018년, 2019년 모두 gdp 정보가 없는 국가 drop 필요
# thresh: 해당 row에서 NaN이 아닌 값이 최소 3개 이상 나와야 한다는 설정
gdp_pop = gdp_df.dropna(axis = 0, thresh = 3)
del gdp_pop['Country Name']
gdp_pop = gdp_pop.reset_index()
del gdp_pop['index']

# gdp_pop[gdp_pop['2019'].isnull()]
# 2019 nan값을 2018 정보로 대체
gdp_pop.iloc[94, 2] = gdp_pop.iloc[94, 1]
gdp_pop.iloc[183, 2] = gdp_pop.iloc[183, 1]
del gdp_pop['2018']

# 데이터 타입 변환
gdp_pop = gdp_pop.astype({'2019':'int64'})
gdp_pop

,country,2019
0,Afghanistan,507
1,Albania,5355
2,Algeria,3975
3,Andorra,40897
4,Angola,2809
...,...,...
181,Vanuatu,3102
182,Viet Nam,2715
183,Yemen,824
184,Zambia,1305


# table join

In [40]:
covid_gdp = pd.merge(df, gdp_pop, how='left', on='country')
covid_gdp

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop,2019
0,Afghanistan,136643,754,5921,72,83849,1263,664045,39828263,507.0
1,Albania,132608,11,2456,0,130053,1,840087,2874554,5355.0
2,Algeria,147883,941,3867,16,102594,584,230861,44664792,3975.0
3,Andorra,14167,12,127,0,13796,9,193595,77393,40897.0
4,Angola,40055,0,939,0,34357,0,669500,33913305,2809.0
...,...,...,...,...,...,...,...,...,...,...
215,Wallis and Futuna,445,0,7,0,438,0,20508,11035,NaN
216,Western Sahara,10,0,1,0,8,0,0,612406,NaN
217,Yemen,6961,9,1366,0,4147,5,127681,30500337,824.0
218,Zambia,178748,2006,2904,37,163681,1442,1977464,18911673,1305.0


In [41]:
# 기존 df(covid)와 gdp df사이의 country 명 일치하지 않는 60개 data 확인 
covid_gdp.isnull().sum()

country        0
tot_cases      0
new_cases      0
tot_deaths     0
new_deaths     0
tot_recov      0
new_recov      0
tests          0
pop            0
2019          60
dtype: int64

# Dataframe 정제1

In [42]:
# 이름 바꿔야하는 국가명 확인
# 60 개
covid_gdp[covid_gdp['2019'].isnull()]

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop,2019
5,Anguilla,111,0,0,0,109,0,24074,15141,NaN
9,Aruba,11169,0,107,0,11029,0,177885,107233,NaN
21,Bermuda,2521,0,33,0,2477,0,333967,62040,NaN
23,Bolivia,455436,1195,17250,31,381945,1445,1933420,11836839,NaN
27,British Virgin Islands,1147,0,3,0,312,0,57201,30438,NaN
28,Brunei,277,0,3,0,256,0,145694,441793,NaN
32,CAR,7142,0,98,0,6859,0,54814,4916079,NaN
37,Caribbean Netherlands,1660,0,17,0,1631,0,12091,26474,NaN
38,Cayman Islands,619,0,2,0,613,0,110733,66516,NaN
40,Channel Islands,5662,0,86,0,4162,0,563229,175513,NaN


df.iloc[0][3]

In [43]:
'''
# excel file 에서 40개 국가명 확인 후 변경
df_1 = df.copy()
df_1.loc[0,"country"] = "United States of America" #usa
df_1.loc[4,"country"] = "Russian Federation" #Russia
df_1.loc[6,"country"] = "United Kingdom" #UK
df_1.loc[12,"country"] = "Iran (Islamic Republic of)" #Iran
df_1.loc[20,"country"] = "Czech Republic" #Czechia
df_1.loc[40,"country"] = "United Arab Emirates" #UAE
df_1.loc[47,"country"] = "Bolivia (Plurinational State of)" Bolivia
#df_1.loc[63,"country"] = "" #Palestine 국가명 못찾음
#df_1.loc[66,"country"] = "Venezuela, Bolivarian Republic of" gdp_pop 미존재
# df_1.loc[67,"country"] = "Egypt, Arab Rep." gdp_pop 미존재
df_1.loc[75,"country"] = "Republic of Moldova" #Moldova
df_1.loc[85,"country"] = "Republic of Korea" #S. Korea
# df_1.loc[88,"country"] = "" #Kyrgyzstan 
df_1.loc[113,"country"] = "Côte D'Ivoire" #Ivory Coast 찾았다....
df_1.loc[116,"country"] = "Democratic Republic of the Congo" #DRC
# df_1.loc[123,"country"] = "" #Réunion 프랑스 섬
df_1.loc[126,"country"] = "Viet Nam" # Vietnam
# df_1.loc[127,"country"] = "" #French Guiana
# df_1.loc[128,"country"] = "Syrian Arab Republic" gdp_pop 미존재
# df_1.loc[135,"country"] = "" # Mayotte
# df_1.loc[137,"country"] = "French Polynesia" 
# df_1.loc[139,"country"] = "" #Guadeloupe
# df_1.loc[141,"country"] = "" # Taiwan
# df_1.loc[151,"country"] = "Curacao" gdp_pop 미존재
# df_1.loc[152,"country"] = "" # Martinique
# df_1.loc[153,"country"] = "Hong Kong SAR, China" gdp_pop 미존재
# df_1.loc[157,"country"] = "Aruba" # 있는데 왜 못찾아? 왜못찾긴 gdp_pop 이랑 비교하라고.....
#df_1.loc[158,"country"] = "Sudan" #sudan 이미 있음
df_1.loc[163,"country"] = "Central African Republic" # CAR
df_1.loc[166,"country"] = "Gambia (Republic of The)" # Gambia
# df_1.loc[167,"country"] = "Eritrea" gdp_pop미존재
# df_1.loc[172,"country"] = "Channel Islands" gdp_pop미존재
# df_1.loc[176,"country"] = "Gibraltar" gdp_pop미존재
df_1.loc[179,"country"] = "Guinea Bissau" # Guinea-Bissau
df_1.loc[182,"country"] = "Lao People’s Democratic Republic" #Laos 찾음
# df_1.loc[183,"country"] = "Sint Maarten (Dutch part)" gdp_pop미존재
# df_1.loc[185,"country"] = "Bermuda" gdp_pop미존재
# df_1.loc[186,"country"] = "Turks and Caicos Islands" gdp_pop미존재
# df_1.loc[188,"country"] = "" # Saint Martin # 카릴브의 섬
df_1.loc[190,"country"] = "Saint Vincent and the Grenadines" # St. Vincent Grenadines
# df_1.loc[192,"country"] = "Isle of Man" #영국령 섬 dpe_pop 미존재 
#df_1.loc[193,"country"] = "" # Caribbean Netherlands #카리브에 있는 네덜란드령
# df_1.loc[195,"country"] = "" # St. Barth #생바르텔레미
#df_1.loc[196,"country"] = "" # Faeroe Islands # 덴마크 왕국 공동체 denmark data 이미 존재 
# df_1.loc[197,"country"] = "British Virgin Islands" gdp_pop 미존재
# df_1.loc[198,"country"] = "Cayman Islands" gdp_pop 미존재
df_1.loc[200,"country"] = "United Republic of Tanzania" # Tanzania
# df_1.loc[201,"country"] = "" # Wallis and Futuna un가입국 아님
df_1.loc[202,"country"] = "Brunei Darussalam" # Brunei
# df_1.loc[205,"country"] = "New Caledonia" gdp_pop 미존재
# df_1.loc[206,"country"] = "" # Anguilla
# df_1.loc[207,"country"] = "" # Falkland Islands
# df_1.loc[208,"country"] = "Macao SAR, China" gdp_pop 미존재
df_1.loc[209,"country"] = "Greenland" # Greenland
# df_1.loc[210,"country"] = "" #Vatican City 바티칸시티
# df_1.loc[211,"country"] = "" # Saint Pierre Miquelon 프랑스 해외 집합체
# df_1.loc[212,"country"] = "" # Montserrat
# df_1.loc[214,"country"] = "" # Western Sahara
#df_1.loc[218,"country"] = "Jordan" # Saint Helena jordan 이미 있음
df_1.loc[219,"country"] = "Micronesia (Federated States of)" #Micronesia

# 21개 데이터의 변경값 찾음
'''

'\n# excel file 에서 40개 국가명 확인 후 변경\ndf_1 = df.copy()\ndf_1.loc[0,"country"] = "United States of America" #usa\ndf_1.loc[4,"country"] = "Russian Federation" #Russia\ndf_1.loc[6,"country"] = "United Kingdom" #UK\ndf_1.loc[12,"country"] = "Iran (Islamic Republic of)" #Iran\ndf_1.loc[20,"country"] = "Czech Republic" #Czechia\ndf_1.loc[40,"country"] = "United Arab Emirates" #UAE\ndf_1.loc[47,"country"] = "Bolivia (Plurinational State of)" Bolivia\n#df_1.loc[63,"country"] = "" #Palestine 국가명 못찾음\n#df_1.loc[66,"country"] = "Venezuela, Bolivarian Republic of" gdp_pop 미존재\n# df_1.loc[67,"country"] = "Egypt, Arab Rep." gdp_pop 미존재\ndf_1.loc[75,"country"] = "Republic of Moldova" #Moldova\ndf_1.loc[85,"country"] = "Republic of Korea" #S. Korea\n# df_1.loc[88,"country"] = "" #Kyrgyzstan \ndf_1.loc[113,"country"] = "Côte D\'Ivoire" #Ivory Coast 찾았다....\ndf_1.loc[116,"country"] = "Democratic Republic of the Congo" #DRC\n# df_1.loc[123,"country"] = "" #Réunion 프랑스 섬\ndf_1.loc[126,"country"] = "Viet Na

In [44]:
# 21개 데이터의 변경값 찾음
df_1 = df.copy()
df_1.loc[206,"country"] = "United States of America" #usa
df_1.loc[160,"country"] = "Russian Federation" #Russia
df_1.loc[205,"country"] = "United Kingdom" #UK
df_1.loc[96,"country"] = "Iran (Islamic Republic of)" #Iran
df_1.loc[51,"country"] = "Czech Republic" #Czechia
df_1.loc[204,"country"] = "United Arab Emirates" #UAE
df_1.loc[23,"country"] = "Bolivia (Plurinational State of)" #Bolivia
df_1.loc[130,"country"] = "Republic of Moldova" #Moldova
df_1.loc[163,"country"] = "Republic of Korea" #S. Korea
df_1.loc[99,"country"] = "Côte D'Ivoire" #Ivory Coast 찾았다....
df_1.loc[52,"country"] = "Democratic Republic of the Congo" #DRC
df_1.loc[214,"country"] = "Viet Nam" # Vietnam
df_1.loc[32,"country"] = "Central African Republic" # CAR
df_1.loc[73,"country"] = "Gambia (Republic of The)" # Gambia
df_1.loc[84,"country"] = "Guinea Bissau" # Guinea-Bissau
df_1.loc[107,"country"] = "Lao People’s Democratic Republic" #Laos 찾음
df_1.loc[188,"country"] = "Saint Vincent and the Grenadines" # St. Vincent Grenadines
df_1.loc[196,"country"] = "United Republic of Tanzania" # Tanzania
df_1.loc[28,"country"] = "Brunei Darussalam" # Brunei
df_1.loc[79,"country"] = "Greenland" # Greenland
df_1.loc[129,"country"] = "Micronesia (Federated States of)" #Micronesia


In [46]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   country     220 non-null    object
 1   tot_cases   220 non-null    int64 
 2   new_cases   220 non-null    int64 
 3   tot_deaths  220 non-null    int64 
 4   new_deaths  220 non-null    int64 
 5   tot_recov   220 non-null    int64 
 6   new_recov   220 non-null    int64 
 7   tests       220 non-null    int64 
 8   pop         220 non-null    int64 
dtypes: int64(8), object(1)
memory usage: 15.6+ KB


# Table join2

In [47]:
covid_gdp_1 = pd.merge(df_1, gdp_pop, how='left', on='country')
covid_gdp_1

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop,2019
0,Afghanistan,136643,754,5921,72,83849,1263,664045,39828263,507.0
1,Albania,132608,11,2456,0,130053,1,840087,2874554,5355.0
2,Algeria,147883,941,3867,16,102594,584,230861,44664792,3975.0
3,Andorra,14167,12,127,0,13796,9,193595,77393,40897.0
4,Angola,40055,0,939,0,34357,0,669500,33913305,2809.0
...,...,...,...,...,...,...,...,...,...,...
215,Wallis and Futuna,445,0,7,0,438,0,20508,11035,NaN
216,Western Sahara,10,0,1,0,8,0,0,612406,NaN
217,Yemen,6961,9,1366,0,4147,5,127681,30500337,824.0
218,Zambia,178748,2006,2904,37,163681,1442,1977464,18911673,1305.0


In [48]:
# 정제 df(covid)와 gdp df사이의 country 명 일치하지 않는 40개 data 확인 
covid_gdp_1.isnull().sum()

country        0
tot_cases      0
new_cases      0
tot_deaths     0
new_deaths     0
tot_recov      0
new_recov      0
tests          0
pop            0
2019          40
dtype: int64

In [49]:
# gdp data는 존재하나 covid data가 존재하지않는 8개 data 존재 확인
covid_gdp_2 = pd.merge(df_1, gdp_pop, how='right', on='country')
covid_gdp_2
covid_gdp_2[covid_gdp_2['tests'].isnull()]
covid_gdp_2.isnull().sum()

country       0
tot_cases     6
new_cases     6
tot_deaths    6
new_deaths    6
tot_recov     6
new_recov     6
tests         6
pop           6
2019          0
dtype: int64

In [50]:
covid_gdp_2[covid_gdp_2['pop'].isnull()]

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop,2019
86,Kiribati,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1655
115,Nauru,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10983
126,Palau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14907
167,Tonga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4903
171,Turkmenistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7612
172,Tuvalu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4055


# Dataframe 정제2

In [ ]:
# nan 값을 갖는 data 확인
covid_gdp_1[covid_gdp_1['2019'].isnull()]

In [ ]:
# column 2019 data==nan 일경우 행 삭제
covid_gdp_1 = covid_gdp_1.dropna(axis=0)

In [ ]:
# 180개의 data 획득
covid_gdp_1.info()

In [ ]:
covid_gdp_1 = covid_gdp_1.astype({'2019':'int64'})
covid_gdp_1.info()

In [ ]:
# final check
covid_gdp_1

In [ ]:
covid_gdp_1.columns = ["country", "tot_cases", "new_cases", "tot_deaths", "new_deaths", "tot_recov", "new_recov", "tests", "pop", "2019"]
covid_gdp_1.to_excel('./covid_gdp.xlsx', index = False)

In [ ]:
covid_gdp_1.info()
